In [39]:
from importlib import reload
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotutils as pu
reload(pu)

%matplotlib widget

In [67]:
def get_utilization(jobs):
    records = [
        (float(row['Started']), float(row['Finished']))
        for _, row in jobs.iterrows()
        if row['Started'] == row['Started'] and row['Finished'] == row['Finished']
    ]
    records.sort()
    sorted_records = []
    def get_position(time):
        if time < sorted_records[0][0]:
            return (-1, False)
        else:
            length = len(sorted_records)
            for i in range(length):
                if time >= sorted_records[i][0] and time <= sorted_records[i][1]:
                    return (i, True)
                elif i < length-1 and time > sorted_records[i][1] and time < sorted_records[i+1][0]:
                    return (i, False)
            if time > sorted_records[length-1][1]:
                return (length-1, False)
            else:
                return (-2, False)
    for record in records:
        if len(sorted_records)==0:
            sorted_records.append(record)
        else:
            start_time = record[0]
            end_time = record[1]
            start_idx, start_wh = get_position(start_time)
            end_idx, end_wh = get_position(end_time)
            if start_idx == end_idx and not start_wh and not end_wh:
                sorted_records.insert(start_idx+1, record)
            elif start_wh:
                new_start = sorted_records[start_idx][0]
                new_end = sorted_records[end_idx][1] if end_wh else end_time
                del sorted_records[start_idx:end_idx+1]
                sorted_records.insert(start_idx, (new_start, new_end))
            else:
                new_start = start_time
                new_end = sorted_records[end_idx][1] if end_wh else end_time
                del sorted_records[start_idx+1:end_idx+1]
                sorted_records.insert(start_idx+1, (new_start, new_end))
    total_time = sorted_records[-1][1] - sorted_records[0][0]
    run_time_list = list(map(lambda x: x[1]-x[0], sorted_records))
    run_time = sum(run_time_list)
    run_per = run_time / total_time
    idle_per = 1 - run_per
    return run_per, idle_per


def show(jobs, figsize=(10,8)):
    fig = plt.figure(figsize=figsize, constrained_layout=True)
    axs = fig.subplot_mosaic('''AC
                                AC
                                BC
                                DC
                                ''')
    # execution
    ax = pu.job_timeline(jobs.JobId, jobs.Admitted/1000, jobs.Deadline/1000, ax=axs['C'], numeric_workers=True)
    done = jobs[jobs.State == 'done']
    ax = pu.job_timeline(done.JobId, done.Started/1000, done.Finished/1000, ax=axs['C'], numeric_workers=True)
    ax.set_title('Execution Timeline')
    ax.set_ylabel('Job')
    ax.set_xlabel('Time') 
    
    # latency CDF
    # for the sake of this CDF, set the latency of requets past due to Nan
    tmp = jobs.copy()
    tmp.loc[tmp.State != 'done', 'Latency'] = np.nan
    ax = pu.cdf(tmp.Latency, ax=axs['A'])
    ax.set_ylabel('CDF')
    ax.set_xlabel('Latency')
    
    # done
    ax = axs['B']
    total = len(jobs)
    states = ['past_due', 'done']
    pos = np.arange(len(states))
    data = [
        len(jobs.State[jobs.State == state]) / total
        for state in states
        ]
    ax.barh(pos, data, align='center')
    for x, y in zip(data, pos):
        pu.bar_show_data(x, y, data_y=x, fmt='{:.0%}', ax=ax, xytext=(4, 0), horizontalalignment='left', verticalalignment='center')
    pu.cleanup_axis_categorical(ax.yaxis, states)
    pu.cleanup_axis_percent(ax.xaxis, xmax=1.0)
    ax.set_xlabel('Percentage')

    # utilization
    ax = axs['D']
    states = ['idle', 'run']
    pos = np.arange(len(states))
    data = get_utilization(jobs)
    ax.barh(pos, data, align='center')
    for x, y in zip(data, pos):
        pu.bar_show_data(x, y, data_y=x, fmt='{:.0%}', ax=ax, xytext=(4, 0), horizontalalignment='left', verticalalignment='center')
    pu.cleanup_axis_categorical(ax.yaxis, states)
    pu.cleanup_axis_percent(ax.xaxis, xmax=1.0)
    ax.set_xlabel('Percentage')

    return fig, axs

In [71]:
clipper = pd.read_csv("final_result.csv")
fig, _ = show(clipper)
fig.suptitle("unit(20)_per(4)_bud(5)")
plt.tight_layout()
fig.savefig("unit(20)_per(4)_bud(5).png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/pleple/opt/anaconda3/envs/clipper/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  after removing the cwd from sys.path.


In [17]:
incoming = pd.read_csv(Path('../log/bs5') / 'fifo' / 'jobs.csv')
outcoming = pd.DataFrame(incoming, columns=["JobId", "Admitted", "Deadline"])
num = incoming.shape[0]
file = open("news.en", "r")
lines = np.array(file.readlines())
total_size = len(lines)
index = np.random.randint(0, total_size, num)
inputs = lines[index]
outcoming["InputSen"] = pd.Series(inputs, index = outcoming.index)
outcoming.to_csv("req.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../log/bs5/fifo/jobs.csv'

In [ ]:
!python async_dynamic_test.py --reqs req.csv --output output.csv

In [ ]:
clipper = incoming.copy().drop(columns=["Length", "LengthP99"])
clipper.head()
clipper_rst = pd.read_csv('output.csv', sep=':')
clipper_rst.loc[clipper_rst["OutputSen"]=="Non", "State"]= "past_due"
vl = clipper_rst.shape[0]
clipper["Latency"] = np.nan
clipper["Started"].iloc[0:vl] = clipper_rst["Timestamp"]
clipper["Latency"].iloc[0:vl] = clipper_rst["LatencyMS"]
clipper["Started"].iloc[vl:] = np.nan
clipper["Latency"].iloc[vl:] = np.nan
clipper["Finished"] = clipper["Started"] + clipper["Latency"]
clipper["State"].iloc[0:vl] = clipper_rst["State"]
clipper["State"].iloc[vl:] = "past_due"
clipper.loc[clipper["Finished"]>clipper["Deadline"], "State"]= "past_due"
clipper.loc[clipper["Finished"]<=clipper["Deadline"], "State"]= "done"
clipper.loc[clipper["State"]=="error", "State"]= "past_due"
clipper.loc[clipper["Finished"]>clipper["Deadline"], "Latency"]= np.nan
clipper.to_csv("final_result.csv", index=False)
clipper.head()

In [ ]:
show(clipper)